In [2]:
from langchain_community.document_loaders import PyPDFLoader
from dotenv import load_dotenv

load_dotenv()
loader = PyPDFLoader("data/act.pdf")
docs = loader.load()

/Users/Supun/Documents/research/ongoing/compliance/compliance/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


llm = ChatOpenAI()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_splitter.split_documents(docs)

vector = FAISS.from_documents(docs, OpenAIEmbeddings(model="text-embedding-3-large"))


prompt = ChatPromptTemplate.from_template("""You are an expert compliance officer in banks in Sri Lanka. Answer the following question based only on the provided context: If you can't find the answer in the context, please respond with "Can not answer that based on this data".

<context>
{context}
</context>

Question: {input}
                                          
                             """)

document_chain = create_stuff_documents_chain(llm, prompt)

In [4]:
from langchain.chains import create_retrieval_chain
retriever = vector.as_retriever(search_kwargs={"k": 3})
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "What is this document?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

This document is the Banking Act (7th Rev.) in Sri Lanka.


In [5]:
from flask import Flask, jsonify, request, json
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/status', methods=['GET'])
def get_status():
    #return status ok
    return jsonify({'status': 'ok'}), 200


#add a POST route at /chat. Get the request body and echo the message back
@app.route('/chat', methods=['POST'])
def chat():
    data = json.loads(request.data)
    
    #get the last message from the user
    message = data['messages'][-1]['content']
    print(message)
    response = retrieval_chain.invoke({"input": message})
    print(response["answer"])
    #message = data['messages']
    return jsonify({'status': 'ok', 'response': response["answer"]}), 200

if __name__ == '__main__':
    app.run(port=3001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3001
Press CTRL+C to quit
127.0.0.1 - - [08/Mar/2024 07:32:58] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 07:32:58] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 07:33:03] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 07:33:03] "GET /status HTTP/1.1" 200 -


What restriction does the Banking Act impose on licensed specialized banks concerning the creation of charges on their unpaid capital?



127.0.0.1 - - [08/Mar/2024 07:33:08] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 07:33:08] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 07:33:09] "POST /chat HTTP/1.1" 200 -


Licensed specialized banks are not allowed to create any charge upon any unpaid capital, and any such charge created in contravention of these provisions shall be null and void.


...

127.0.0.1 - - [08/Mar/2024 09:01:25] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:25] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:30] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:30] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:35] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:35] "GET /status HTTP/1.1" 200 -


which act?


127.0.0.1 - - [08/Mar/2024 09:01:40] "POST /chat HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:40] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:40] "GET /status HTTP/1.1" 200 -


The act referred to in the context is the Banking Act (7th Rev.) of Sri Lanka.


127.0.0.1 - - [08/Mar/2024 09:01:45] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:45] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:50] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:50] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:55] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:01:55] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:00] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:00] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:05] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:05] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:14] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:14] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:19] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:19] "GET /status HTTP/1.1" 200 -
127.0.0.1 - - [08/Mar/2024 09:02:24] "GET /status HTTP/1.1" 200 -
127.0.0.1 